## Dependencies

---

In [39]:
import pandas as pd

## Pre-loading

---

### Functions:

In [40]:
def add_uid(func):
    """
    Add uid to the dimension created.
    :param func: function that creates dimensions.
    :return: the dimension created by the function + an uid.
    """
    def wrapper(**kargs) -> pd.DataFrame:
        dim = func(**kargs)
        dim[f"{kargs['column']}_id"] = dim.index + 1
        return dim
    return wrapper

In [41]:
def create_dim(column:str, dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Extract the feature and parse it to a dataframe.
    :param column: feature to extract.
    :param dataframe: object where feature will be extracted.
    :return: dataframe with the feature and its new column.
    """
    dim = dataframe[f"{column}"].to_frame()
    return dim

### Dimensions & facts:

First of all is required to split the dataframes in `Dimensions`, where they can be related with the `Facts` by an unique id.

In [42]:
df_temp = pd.read_csv("persistence/sedan-cards_march-april.csv")

In [48]:
columns = df_temp.columns.to_list()
dimensions_keys = ["model", "description", "engine"]
facts_keys = ["year", "kilometers", "price"]

In [49]:
create_dim_uid = add_uid(create_dim)
dimensions_df = []
facts_df = []

In [50]:
for column in columns:
    if column in dimensions_keys:
        dim = create_dim_uid(column=column, dataframe=df_temp)
        dimensions_df.append(dim)
    elif column in facts_keys:
        fact = create_dim_uid(column=column, dataframe=df_temp)
        facts_df.append(fact)

In [57]:
dim_model = dimensions_df[0]
dim_engine = dimensions_df[1]
dim_description = dimensions_df[2]

dim_facts = pd.concat(facts_df, axis=1)
dim_facts = dim_facts.rename(columns={"year_id":"model_id","kilometers_id":"engine_id","price_id":"description_id"})